Requirement
=============


------------

From this url `http://www.5metal.com.hk/ajax/pager/company_fea?view_amount=36&page=1`,

We get a list of urls for detail pages like `http://www.5metal.com.hk/wongwahkee/contact` to scrape the companies' info

------------


Info to scrape:

1. address
2. company name
3. company website
4. contact number
5. contact person
6. email
7. mobile phone

------------

In [1]:
from IPython.core.display import display, HTML
import urllib2
import bs4
import urlparse
import pandas as pd

Observation
------------

Since the pagination of the website is crappy (or convenient), you can actually set `view_count` to a super large number to **avoid handling pagination issue**, also note that the `page` parameter is **zero-indexed**

In [2]:
index_page_url = 'http://www.5metal.com.hk/ajax/pager/company_fea?view_amount=9999999'

In [3]:
index_page_src = urllib2.urlopen(index_page_url).read()
index_page_soup = bs4.BeautifulSoup(index_page_src, 'html.parser')

The index page HTML source:

In [4]:
print(index_page_soup.prettify())

<div class="view view-company-list view-id-company_list view-display-id-block_1 view-dom-id-029ac74089cfb9da30fbadd5a6fcabc2">
 <div class="view-content">
  <div>
   <div class="views-field views-field-nothing">
    <span class="field-content">
     <div class="company">
      <div class="logo">
       <a href="/node/10624" target="_blank">
        <img alt="" height="113" src="https://www.5metal.com.hk/sites/default/files/styles/company_list_logo/public/logo%20my%20home.jpg?itok=IJL9ebcH" width="152"/>
       </a>
      </div>
      <div class="title">
       <a href="/node/10624" target="_blank">
        愛家潔具有限公司
       </a>
      </div>
     </div>
    </span>
   </div>
  </div>
  <div>
   <div class="views-field views-field-nothing">
    <span class="field-content">
     <div class="company">
      <div class="logo">
       <a href="/node/13112" target="_blank">
        <img alt="" height="113" src="https://www.5metal.com.hk/sites/default/files/styles/company_list_logo/public/Kwong

The index page looks like:

In [5]:
display(HTML(index_page_src))

Thank you for the nice website design =3

In [6]:

detail_pages_links = [anchor.get('href') for anchor in index_page_soup.select('.views-field .title a[href]')]
detail_pages_links = [urlparse.urljoin(index_page_url, link) for link in detail_pages_links if link is not None]
scraped_data = pd.DataFrame(data={'detail_page_url': detail_pages_links})

print('No. of links got: ' + str(scraped_data.shape[0]))
print(scraped_data)

No. of links got: 36
                        detail_page_url
0   http://www.5metal.com.hk/node/10624
1   http://www.5metal.com.hk/node/13112
2       http://www.5metal.com.hk/node/1
3    http://www.5metal.com.hk/node/3869
4    http://www.5metal.com.hk/node/8422
5    http://www.5metal.com.hk/node/3652
6     http://www.5metal.com.hk/node/149
7   http://www.5metal.com.hk/node/14513
8    http://www.5metal.com.hk/node/6018
9   http://www.5metal.com.hk/node/15998
10   http://www.5metal.com.hk/node/3563
11  http://www.5metal.com.hk/node/13379
12   http://www.5metal.com.hk/node/8507
13   http://www.5metal.com.hk/node/3568
14   http://www.5metal.com.hk/node/3705
15  http://www.5metal.com.hk/node/18680
16   http://www.5metal.com.hk/node/3594
17   http://www.5metal.com.hk/node/5311
18  http://www.5metal.com.hk/node/20190
19   http://www.5metal.com.hk/node/5334
20  http://www.5metal.com.hk/node/13707
21   http://www.5metal.com.hk/node/7106
22  http://www.5metal.com.hk/node/14179
23   http://www.5me